In [1]:
from common import *
from pandas.api.types import CategoricalDtype
from numba import njit

from sklearn.preprocessing import QuantileTransformer as QT
from scipy.stats import norm
ppf = norm.ppf

In [2]:
from given import *


# ------------------------------> looking at `W`

In [3]:
#pd.to_pickle((F,P), '/big/data/saves/train_5searchbase.32.pkl')
F,P = pd.read_pickle('/big/data/saves/train_5fixedsince+1.32.pkl')

### Scratcher

In [ ]:
F.index = P.index = F.index.map(lambda x: '{:x}'.format(x))

In [68]:
from sklearn.model_selection import GroupShuffleSplit

ho = P.quarter>=2015.5
ho.name = None
tr, cv = next(GroupShuffleSplit(n_splits=5, test_size=.5, random_state=44).split(F[~ho], P[~ho], groups=P[~ho].quarter))
_dummy = pd.Series(range(len(P)),index=P.index)
#tr, cv = _dummy.isin(tr), _dummy.isin(cv)

In [71]:
F.index[tr]

Index(['b2e8c', 'b2e8d', 'b2e8e', 'b2e8f', 'b2e90', 'b2e91', 'b2e92', 'b2e93',
       'b2e94', 'b2e95',
       ...
       '321a76', '321a77', '321a78', '321a79', '321a7a', '321a7b', '321a7c',
       '321a7d', '321a7e', '321a7f'],
      dtype='object', length=1346646)

In [72]:
guy = GroupShuffleSplit(n_splits=6, test_size=.5, random_state=44).split(F[~ho], P[~ho], groups=P[~ho].quarter)
for tr, cv in guy:
    plist(P.quarter[_dummy.isin(tr)].unique(), end=',\n')
    plist(P.quarter[_dummy.isin(cv)].unique(), end='\n\n')

[2009.0,2009.75,2010.75,2011.5,2012.25,2012.5,2012.75,2013.0,2013.25,2013.75,2014.0,2014.25,2015.0],
[2009.25,2009.5,2010.0,2010.25,2010.5,2011.0,2011.25,2011.75,2012.0,2013.5,2014.5,2014.75,2015.25]

[2009.25,2009.5,2011.0,2011.25,2011.5,2011.75,2012.0,2012.25,2013.0,2013.5,2014.5,2014.75,2015.0],
[2009.0,2009.75,2010.0,2010.25,2010.5,2010.75,2012.5,2012.75,2013.25,2013.75,2014.0,2014.25,2015.25]

[2011.0,2011.75,2012.0,2012.25,2012.75,2013.25,2013.5,2014.0,2014.25,2014.5,2014.75,2015.0,2015.25],
[2009.0,2009.25,2009.5,2009.75,2010.0,2010.25,2010.5,2010.75,2011.25,2011.5,2012.5,2013.0,2013.75]

[2009.5,2010.0,2010.25,2010.5,2011.0,2011.75,2012.25,2012.5,2013.0,2013.5,2014.25,2014.75,2015.25],
[2009.0,2009.25,2009.75,2010.75,2011.25,2011.5,2012.0,2012.75,2013.25,2013.75,2014.0,2014.5,2015.0]

[2009.75,2010.0,2010.25,2010.75,2011.75,2012.0,2012.25,2012.5,2013.0,2013.5,2013.75,2014.5,2015.0],
[2009.0,2009.25,2009.5,2010.5,2011.0,2011.25,2011.5,2012.75,2013.25,2014.0,2014.25,2014.75,2015.

### stock subsets

In [24]:
Mt = M[M.time>=train_start_time]

In [12]:
# # already in given.ipynb
# inUniCount = Mt.groupby('assetCodeId').universe.sum()
# stocksInUni = (inUniCount != 0).pipe(lambda x: x.index[x])
# assert stocksInUni.is_monotonic
# xStocksInUni = set(stocksInUni) #EDITCELL

In [25]:
W = O()
W.y1 = Mt[['time','assetCodeId','oo']].pivot('time','assetCodeId').oo
W.ay1 = Mt[['time','assetCodeId','aoo']].pivot('time','assetCodeId').aoo
W.y10 = Mt[['time','assetCodeId','ooTEN']].pivot('time','assetCodeId').ooTEN
W.ay10 = Mt[['time','assetCodeId','aooTEN']].pivot('time','assetCodeId').aooTEN
W.u = Mt[['time','assetCodeId','universe']].pivot('time','assetCodeId').universe.fillna(0).astype(int)

In [27]:
C = O()
C.y1 = W.y1.corr()
C.ay1 = W.ay1.corr()
C.y10 = W.y10.corr()
C.ay10 = W.ay10.corr()
C.u = W.u.T @ W.u

In [28]:
@njit
def offdiag_items(ix, x, y):
    m, n = x.shape
    assert (m, n) == y.shape
    assert m == n
    assert (m,) == ix.shape
    t = (n**2-n) // 2
    I = np.full(t, -1, dtype=np.int64)
    J = np.full(t, -1, dtype=np.int64)
    X = np.full(t, 0, dtype=x.dtype)
    Y = np.full(t, 0, dtype=y.dtype)
    k = -1
    for i in range(m):
        for j in range(i+1, n):
            k += 1
            I[k] = ix[i]
            J[k] = ix[j]
            X[k] = x[i,j]
            Y[k] = y[i,j]
    return I, J, X, Y

In [29]:
C.v1 = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.y1.index.values, C.y1.values, C.u.values))))
C.av1 = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.ay1.index.values, C.ay1.values, C.u.values))))
C.v10 = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.y10.index.values, C.y10.values, C.u.values))))
C.av10 = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.ay10.index.values, C.ay10.values, C.u.values))))

In [30]:
pd.to_pickle((O.py(W),O.py(C)), big_data/'pair/working_2ONEorTEN.pkl')
#(W, C) = pd.read_pickle(big_data/'pair/working_2ONEorTEN.pkl'); W, C = O(**W), O(**C)

In [43]:
for c in ['v1','av1','v10','av10']:
    C[c] = C[c][C[c].Unic!=0]
    C[c].sort_values('Corr', ascending=False, inplace=True)

*see some stuff*

In [26]:
C.v[C.av.Unic>=1900].sort_values('Corr', ascending=False).Corr.pipe(lambda x: sum(x>.65))

1523

In [31]:
assetNameIdAssign.series.loc[3741], assetNameIdAssign.series.loc[3759]

('Syneos Health Inc', 'Varonis Systems Inc')

#### making pair feature table

In [9]:
def make_E_from_F_P(*, F, P):
    # add stuff to F I need
    F['time'] = P.time
    F['universe'] = P.universe # not removed because need it for generating pair features
    F['assetCodeIdPivot'] = F.assetCodeId.astype(int)
    # the pivot ===============\___
    E = F.pivot('time','assetCodeIdPivot')
    # remove stuff I added to F
    del F['time'], F['universe'], F['assetCodeIdPivot']
    # end block
    
    E.columns = E.columns.swaplevel()
    E = E.sort_index(axis=1)
    E.columns.rename('assetCodeId', level=0, inplace=True)
    return E

In [10]:
E = make_E_from_F_P(F=F[['assetCodeId','oo1','oo10','oo(15-5)','aoo1','aoo10','aoo(15-5)']], P=P)
E.iloc[:12,:12]

assetCodeIdPivot                  0                                            \
                          aoo(15-5)      aoo1     aoo10 assetCodeId  oo(15-5)   
time                                                                            
2009-01-02 22:00:00+00:00 -0.122458  0.041259 -0.015430         0.0 -0.144766   
2009-01-05 22:00:00+00:00 -0.107106  0.022029 -0.009322         0.0 -0.133051   
2009-01-06 22:00:00+00:00 -0.125086  0.058453  0.043500         0.0 -0.123945   
2009-01-07 22:00:00+00:00 -0.133125  0.043501  0.084718         0.0 -0.141775   
2009-01-08 22:00:00+00:00 -0.103358  0.017562  0.108553         0.0 -0.095188   
2009-01-09 22:00:00+00:00 -0.015430  0.007358  0.134983         0.0 -0.037740   
2009-01-12 22:00:00+00:00 -0.009322  0.034463  0.172058         0.0 -0.019048   
2009-01-13 22:00:00+00:00  0.043500 -0.027127  0.155250         0.0  0.050370   
2009-01-14 22:00:00+00:00  0.084718  0.011800  0.189917         0.0  0.090838   
2009-01-15 22:00:00+00:00  0.108553  0.012108  0.221406         0.0  0.124759   
2009-01-16 22:00:00+00:00  0.134983  0.052071  0.232218         0.0  0.160176   
2009-01-20 22:00:00+00:00  0.172058  0.023928  0.234117         0.0  0.189772   

assetCodeIdPivot                                               2            \
                                oo1      oo10 universe aoo(15-5)      aoo1   
time                                                                         
2009-01-02 22:00:00+00:00  0.044568 -0.037740      1.0 -0.016283  0.009114   
2009-01-05 22:00:00+00:00  0.032790 -0.019048      1.0 -0.003004 -0.026058   
2009-01-06 22:00:00+00:00  0.059018  0.050370      1.0  0.009410  0.001343   
2009-01-07 22:00:00+00:00  0.042925  0.090838      1.0 -0.011282 -0.016710   
2009-01-08 22:00:00+00:00  0.017218  0.124759      1.0  0.037926 -0.007776   
2009-01-09 22:00:00+00:00  0.008226  0.160176      1.0  0.044710 -0.004003   
2009-01-12 22:00:00+00:00  0.029596  0.189772      1.0 -0.008330 -0.003935   
2009-01-13 22:00:00+00:00 -0.035622  0.163166      1.0 -0.026816 -0.005824   
2009-01-14 22:00:00+00:00  0.010929  0.197656      1.0 -0.015188 -0.004064   
2009-01-15 22:00:00+00:00  0.003797  0.213445      1.0  0.002489  0.013764   
2009-01-16 22:00:00+00:00  0.052727  0.221604      1.0  0.006364  0.027048   
2009-01-20 22:00:00+00:00  0.027357  0.216171      1.0 -0.016645  0.001083   

assetCodeIdPivot                                 
                              aoo10 assetCodeId  
time                                             
2009-01-02 22:00:00+00:00  0.044710         2.0  
2009-01-05 22:00:00+00:00 -0.008330         2.0  
2009-01-06 22:00:00+00:00 -0.026816         2.0  
2009-01-07 22:00:00+00:00 -0.015188         2.0  
2009-01-08 22:00:00+00:00  0.002489         2.0  
2009-01-09 22:00:00+00:00  0.006364         2.0  
2009-01-12 22:00:00+00:00 -0.016645         2.0  
2009-01-13 22:00:00+00:00 -0.022264         2.0  
2009-01-14 22:00:00+00:00 -0.030293         2.0  
2009-01-15 22:00:00+00:00 -0.044150         2.0  
2009-01-16 22:00:00+00:00 -0.026216         2.0  
2009-01-20 22:00:00+00:00  0.000925         2.0

In [46]:
C.av1.head(400)

,0,1,Corr,Unic
2450496,941,3476,1.000000,3
1934188,717,2727,1.000000,3
493806,166,2573,1.000000,2
4620071,2112,2905,1.000000,2
4321676,1924,2905,1.000000,4
1807976,668,2905,1.000000,4
4955195,2335,2905,1.000000,4
1267691,457,2905,1.000000,4
3146963,1246,2905,1.000000,4
5550109,2833,2958,1.000000,3


In [59]:
def make_D_L_from_E_Cv(*, E, Cv, cutoff, roll_corr=list(product(['oo','ooTEN'],[10,21,62,250]))):
    Cv = Cv[(Cv.Corr>=cutoff[0])&(Cv.Unic>=cutoff[1])]
    hgt = sum(p[-1] for p in Cv.itertuples())
    Fcolumns = E.columns.levels[1]
    D0, D1, L = np.zeros((hgt,len(Fcolumns)-1)), np.zeros((hgt,len(Fcolumns)-1)), np.zeros((hgt,1+len(roll_corr)))
    #^ -1 on len(Fcolumns) because I take away the 'universe' column
    
    i = 0
    for _,a0,a1,corr,unic in Cv.itertuples():
        
        A0, A1 = E[a0], E[a1]
        inUniD = (A0.universe!=0) & (A1.universe!=0)
        print(inUniD.sum(), unic)
        assert inUniD.sum()==unic
        del A0['universe'], A1['universe']
        
        def iter_roll_corr():
            for y, r in roll_corr:
                yield A0[y].rolling(window=r, min_periods=5).corr(A1[y]) #TODO hard coded min_periods
        B = np.concatenate(chain(iter_roll_corr(),[np.ones_like(A0)*unic]), axis=1)
        
        D0[i:i+unic], D1[i:i+unic] = A0[inUniD].values, A1[inUniD].values
        L[i:i+unic] = B[inUniD]
        
        i += unic
        
    D0 = pd.DataFrame(D0, columns=Fcolumns)
    D1 = pd.DataFrame(D1, columns=Fcolumns)
    L = pd.DataFrame(L, columns=[f'_corr_{y}_{r}' for y,r in roll_corr]+['bothInUniCount'])
    return (D0,D1), L

In [60]:
D, L = make_D_L_from_E_Cv(E=E,Cv=C.av1,cutoff=(.9,1500))

2014 1555


AssertionError: 

# ------------------------------> looking at `N`